## Algorithm from the book: longest common subsequence

In [1]:
def lcs(xs, ys): 
    m, n = len(xs), len(ys) 
    d = [[0] * (n + 1) for _ in range(m + 1)]
    b = [[None] * n for _ in range(m)]
  
    for i in range(1, m + 1): 
        for j in range(1, n + 1): 
            if xs[i - 1] == ys[j - 1]: 
                d[i][j] = d[i - 1][j - 1] + 1
                b[i - 1][j - 1] = "d"
            elif d[i - 1][j] >= d[i][j - 1]:
                d[i][j] = d[i - 1][j]
                b[i - 1][j - 1] = "u"
            else:
                d[i][j] = d[i][j - 1]
                b[i - 1][j - 1] = "b"
  
    return d[m][n], b


def print_lcs(b, xs, i, j):
    if i == -1 or j == -1:
        return
    if b[i][j] == "d":
        print_lcs(b, xs, i - 1, j - 1)
        print(xs[i], end="")
    elif b[i][j] == "u":
        print_lcs(b, xs, i - 1, j)
    else:
        print_lcs(b, xs, i, j - 1)

In [2]:
X = "XAAYAAAAZA"
Y = "BBXBYBBBZ"
res = lcs(X, Y)

print("The length of the LCS is", res[0])
print("LCS: ", end="")
print_lcs(res[1], X, len(X) - 1, len(Y) - 1)

The length of the LCS is 3
LCS: XYZ

## Algorithm from research paper:  AdaBelief

In [3]:
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize

In [4]:
# loading data from task 3 to compare results

alpha = np.load("../task3/data/alpha.npy")[0]
beta = np.load("../task3/data/beta.npy")[0]
print(f"alpha = {alpha}")
print(f"beta = {beta}")

xs = np.load("../task3/data/xs.npy")[0].tolist()
ys = np.load("../task3/data/ys.npy")[0].tolist()
n = len(xs)

alpha = 0.4591910887680559
beta = 0.0379706926337019


In [5]:
def linear(x, a, b):
    return a * x + b

def rational(x, a, b):
    return a / (1 + b * x)

grad = {
    linear: lambda x, a, b: (x, 1),
    rational: lambda x, a, b: (1 / (b * x + 1), a * x / (b * x + 1) ** 2)
}

In [6]:
def D(x, f, reduce="sum"):    
    a, b = x
    result = np.array([(f(x, a, b) - y) ** 2 for x, y in zip(xs, ys)])
    if reduce == "sum":
        return result.sum()
    elif reduce == "mean":
        return result.mean()
    elif reduce == "none":
        return result
    else:
        raise Exception

def get_gradient(x, f, reduce="mean"):    
    a, b = x
    g1 = np.array([2 * grad[f](x, a, b)[0] * (f(x, a, b) - y) for x, y in zip(xs, ys)])
    g2 = np.array([2 * grad[f](x, a, b)[1] * (f(x, a, b) - y) for x, y in zip(xs, ys)])
    
    if reduce == "sum":
        return np.array([g1.sum(), g2.sum()])
    elif reduce == "mean":
        return np.array([g1.mean(), g2.mean()])
    elif reduce == "none":
        return np.stack([g1, g2]).reshape(n, -1)
    else:
        raise Exception

In [7]:
def adabelief(f, x0, betas=(0.9, 0.999), lr=0.1, k=0.5, max_iters=100, tol=0.001, eps=1e-8):
    a, b = x0
    b1, b2 = betas
    
    a_prev = b_prev = float("inf")
    t = 0
    m = np.array([0, 0])
    s = np.array([0, 0])
    loss = []
    while t < max_iters and math.sqrt((a - a_prev) ** 2 + (b - b_prev) ** 2) > tol:
        t += 1
        a_prev, b_prev = a, b
        
        # calculate gradient
        g1, g2 = get_gradient((a, b), f)
        g = np.array([g1, g2])
        
        m = b1 * m + (1 - b1) * g
        s = b2 * s + (1 - b2) * (g - m) ** 2
    
        # update parameters
        a -= lr * m[0] / (np.sqrt(s[0]) + eps)
        b -= lr * m[1] / (np.sqrt(s[1]) + eps)
        
        loss.append(D((a, b), f))
        
        # update learning rate
        lr *= 1 / (1 + k * t)  

    return (a, b), t, loss

In [8]:
result = adabelief(linear, (-0.05, 0.05), lr=0.1, k=0.03)
a, b = result[0]

print("Linear approximant:")
print(f"fun: {D((a, b), linear)}")
print(f"nit: {result[1]}")
print(f"  x: {[a, b]}")

Linear approximant:
fun: 87.20763531929803
nit: 21
  x: [0.05632552682610621, 0.33733165259521936]


In [9]:
result = adabelief(rational, (-0.05, 0.05), lr=0.055, k=0.85)
a, b = result[0]

print("Rational approximant:")
print(f"fun: {D((a, b), rational)}")
print(f"nit: {result[1]}")
print(f"  x: {[a, b]}")

Rational approximant:
fun: 87.22589704852011
nit: 6
  x: [0.350786471789178, -0.05833229794727621]
